In [362]:
%pip install pandas
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 1000)

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#https://stats.oecd.org/SDMX-JSON/data/CITIES/AUS+AUS01+AUS02+AUS03+AUS04.POP_DEN_HINTER/all?startTime=2019&endTime=2021&contentType=csv&csv This url works
#The following API is for all USA metro areas in OECD cities database. 
urlmetro = 'https://stats.oecd.org/SDMX-JSON/data/CITIES/USA+USA01+USA02+USA03+USA04+USA05+USA06+USA07+USA08+USA09+USA10+USA11+USA12+USA13+USA14+USA15+USA16+USA17+USA18+USA19+USA20+USA21+USA22+USA23+USA24+USA25+USA26+USA27+USA28+USA29+USA30+USA31+USA32+USA33+USA34+USA35+USA36+USA37+USA38+USA39+USA40+USA41+USA42+USA43+USA44+USA45+USA46+USA47+USA48+USA49+USA50+USA51+USA52+USA53+USA54+USA55+USA56+USA57+USA58+USA59+USA60+USA61+USA62+USA63+USA64+USA65+USA66+USA67+USA68+USA69+USA70+USA71+USA72+USA73+USA74+USA75+USA76+USA77+USA78+USA79+USA80+USA81+USA82+USA83+USA84+USA85+USA86+USA87+USA88+USA89+USA90+USA91+USA92+USA93+USA94+USA95+USA96+USA97+USA98+USA99+USA100+USA101+USA102+USA103+USA104+USA105+USA106+USA107+USA108+USA109+USA110+USA111+USA112+USA113+USA114+USA115+USA116+USA117+USA118+USA119+USA120+USA121+USA122+USA123+USA124+USA125+USA126+USA127+USA128+USA129+USA130+USA131+USA132+USA133+USA134+USA135+USA136+USA137+USA138+USA139+USA140+USA141+USA142+USA143+USA144+USA145+USA146+USA147+USA148+USA149+USA150+USA151+USA152+USA153+USA154+USA155+USA156+USA157+USA158+USA160+USA161+USA162+USA170.POP_DEN+T_T+POP_TOT_GI+T_T_SH_NAT+T_Y15_64+EMP_RA_15_MAX+EMP_RA_15_MAX_GR_2007+GDP_REAL_PPP+GDP_SH_NAT+GDP_PC_REAL_PPP+EMP_IND_TOTAL+GDP_PW_REAL_PPP+XDSL+SPEX_25_CORE+TREECOVER_SHARE_CORE/all?startTime=2000&endTime=2021&contentType=csv&csv'
dfmetro = pd.read_csv(urlmetro)

In [ ]:
#Tidy up the column names and specific columns to make it easier to conduct analysis.
import matplotlib as plt
dfmetro.columns = dfmetro.columns.str.lower()
dfmetro.columns = dfmetro.columns.str.replace(' ', '_')
dfmetro.variables = dfmetro.variables.str.lower()
dfmetro.variables = dfmetro.variables.str.replace(' ', '_')
dfmetro.metropolitan_areas = dfmetro.metropolitan_areas.str.replace('(Greater)','')
dfmetro.metropolitan_areas = dfmetro.metropolitan_areas.str.replace(r"\(.*?\)","")
dfmetro.metropolitan_areas = dfmetro.metropolitan_areas.str.replace('[\([{})\]]','')
dfmetro.metropolitan_areas = dfmetro.metropolitan_areas.str.replace('-','')
dfmetro.metropolitan_areas = dfmetro.metropolitan_areas.str.replace(' ','')
dfmetro.metropolitan_areas = dfmetro.metropolitan_areas.str.replace('[\([{})\]]','')
dfmetro.metropolitan_areas = dfmetro.metropolitan_areas.str.lower()


#print(dfmetro.variables.unique())
h = dfmetro[(dfmetro['variables'] == 'working_age_population_group_(15-64)') & (dfmetro.year == 2018)]
h.hist(column='value',bins=100) #This shows that population density is not normally distributed with most of the data on the left handside of the distribution.
print(h.value.mean()) #The mean population density is 167.1
print(h.value.median()) #The median population density is 126.5
#WB data for the USA shows that population density (population per km2 of land) in 2020 was 36. Therefore, this dataset of cities is significantly larger than the country as a whole.
dfmetro.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
x = dfmetro[(dfmetro['variables'] == 'growth/shrinking_index_of_the_total_population_(2001=100)') & (dfmetro.year == 2019)]
x['diff_avg'] = ((x['value'] - x.value.median())/x.value.median())
y = dfmetro[(dfmetro['variables'] == 'labour_productivity_(gdp_per_worker_in_usd,_constant_prices,_constant_ppp,_base_year_2015)') & (dfmetro.year == 2019)]
y['diff_avg'] = ((y['value'] - y.value.mean())/y.value.median())
mdf = pd.merge(x,y,left_on='metro_id',right_on='metro_id')
mdf = mdf.sort_values('diff_avg_x',ascending=False)


from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

model.fit(mdf[['value_x']], mdf[['diff_avg_y']])

#xfit = np.linspace(-1, 7, 162)
#yfit = model.predict(mdf[['value_x']])

#plt.scatter(mdf['value_x'],mdf['diff_avg_y'])
#plt.plot(xfit, yfit,color='red')
sns.regplot(data=mdf,x='value_x',y='diff_avg_y')
mdf.metropolitan_areas_x.sort_values(ascending=True).unique()


In [349]:
dfeduc = pd.read_excel(r'C:\Users\vw191\OneDrive - University of Bristol\Course\3rd year\Data Science\Project\Datastore\US_educationa_attainment_metro.xlsx')
dfeduc.columns = dfeduc.columns.str.lower()
dfeduc.columns = dfeduc.columns.str.replace(' ','_')
dfeduc[['name','state_abbrv']] = dfeduc['name'].str.split(',',1,expand=True)
dfeduc.name = dfeduc.name.str.replace('-','')
dfeduc.name = dfeduc.name.str.replace(' ','')
dfeduc.name = dfeduc.name.str.lower()
dfeduc.name.sort_values(ascending=True).unique()


array(['abilene', 'aguadillaisabela', 'akron', 'albany',
       'albanyschenectadytroy', 'albuquerque', 'alexandria',
       'allentownbethlehemeaston', 'altoona', 'amarillo', 'ames',
       'anchorage', 'annarbor', 'annistonoxfordjacksonville', 'appleton',
       'asheville', 'athensclarkecounty', 'atlantasandyspringsroswell',
       'atlanticcityhammonton', 'auburnopelika', 'augustarichmondcounty',
       'austinroundrock', 'bakersfield', 'baltimorecolumbiatowson',
       'bangor', 'barnstabletown', 'batonrouge', 'battlecreek', 'baycity',
       'beaumontportarthur', 'beckley', 'bellingham', 'bendredmond',
       'billings', 'binghamton', 'birminghamhoover', 'bismarck',
       'blacksburgchristiansburgradford', 'bloomington',
       'bloomsburgberwick', 'boisecity', 'bostoncambridgenewton',
       'boulder', 'bowlinggreen', 'bremertonsilverdale',
       'bridgeportstamfordnorwalk', 'brownsvilleharlingen', 'brunswick',
       'buffalocheektowaganiagarafalls', 'burlington',
       'bur

In [ ]:
mmdf = pd.merge(mdf,dfeduc,left_on='metropolitan_areas_x',right_on='name')
mmdf

In [342]:
#The best way to work with an api from Bea is to use the xml format over the json format.
#This is because vegalite cannot read large numbers in the json format of the api because there are commas in the numbers.
#Therefore I found it better to read the data in an xml format in Jupyter notebook and then conduct the analysis before taking the data into vegalite.

#https://apps.bea.gov/api/data/?UserID=F88DCBE6-81B9-4A33-BDD9-F9B983466DEF&method=GetParameterValuesFiltered&datasetname=Regional&TargetParameter=LineCode&TableName=CAGDP1&ResultFormat=xml


#CAGDP1-3 GDP in current USD.
df = pd.read_xml('https://apps.bea.gov/api/data/?UserID=F88DCBE6-81B9-4A33-BDD9-F9B983466DEF&method=GetData&datasetname=Regional&TableName=CAGDP1&LineCode=3&Year=2018&GeoFips=MSA&ResultFormat=xml',xpath="//Data") #Used api with apikey to get data from Bea. Needs to get Data in the element tree.
df
#CAINC30-100 Total population.
df1 = pd.read_xml('https://apps.bea.gov/api/data/?UserID=F88DCBE6-81B9-4A33-BDD9-F9B983466DEF&method=GetData&datasetname=Regional&TableName=CAINC30&LineCode=100&Year=2018&GeoFips=MSA&ResultFormat=xml',xpath="//Data") #Used api with apikey to get data from Bea. Needs to get Data in the element tree.
df1
#CAINC30-190 Wages and Salaries.
df2 = pd.read_xml('https://apps.bea.gov/api/data/?UserID=F88DCBE6-81B9-4A33-BDD9-F9B983466DEF&method=GetData&datasetname=Regional&TableName=CAINC30&LineCode=190&Year=2018&GeoFips=MSA&ResultFormat=xml',xpath="//Data") #Used api with apikey to get data from Bea. Needs to get Data in the element tree.
df2
#CAINC30-300 Average Wages and Salaries.
df3 = pd.read_xml('https://apps.bea.gov/api/data/?UserID=F88DCBE6-81B9-4A33-BDD9-F9B983466DEF&method=GetData&datasetname=Regional&TableName=CAINC30&LineCode=300&Year=2018&GeoFips=MSA&ResultFormat=xml',xpath="//Data") #Used api with apikey to get data from Bea. Needs to get Data in the element tree.
df3
#CAINC30-300 Average Wages and Salaries.
df4 = pd.read_xml('https://apps.bea.gov/api/data/?UserID=F88DCBE6-81B9-4A33-BDD9-F9B983466DEF&method=GetData&datasetname=Regional&TableName=CAINC30&LineCode=300&Year=2018&GeoFips=MIC&ResultFormat=xml',xpath="//Data", encoding='cp1252') #Used api with apikey to get data from Bea. Needs to get Data in the element tree.
df4

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,CAINC30-300,10100,"Aberdeen, SD (Micropolitan Statistical Area)",2018,Dollars,0,"42,252",None
1,CAINC30-300,10140,"Aberdeen, WA (Micropolitan Statistical Area)",2018,Dollars,0,"41,710",None
2,CAINC30-300,10220,"Ada, OK (Micropolitan Statistical Area)",2018,Dollars,0,"44,309",None
3,CAINC30-300,10300,"Adrian, MI (Micropolitan Statistical Area)",2018,Dollars,0,"44,210",None
4,CAINC30-300,10460,"Alamogordo, NM (Micropolitan Statistical Area)",2018,Dollars,0,"43,876",None
...,...,...,...,...,...,...,...,...
538,CAINC30-300,49300,"Wooster, OH (Micropolitan Statistical Area)",2018,Dollars,0,"46,472",None
539,CAINC30-300,49380,"Worthington, MN (Micropolitan Statistical Area)",2018,Dollars,0,"43,381",None
540,CAINC30-300,49460,"Yankton, SD (Micropolitan Statistical Area)",2018,Dollars,0,"43,023",None
541,CAINC30-300,49780,"Zanesville, OH (Micropolitan Statistical Area)",2018,Dollars,0,"41,945",None


In [392]:
#For loop to clean data for all df from Bea API.
for z in [df, df1, df2, df3]:
    i = z
    i.columns = i.columns.str.lower()
    i.columns = i.columns.astype('str').str.replace(' ','_')
    i.datavalue = i.datavalue.astype('str').str.replace(",",'')
    i.datavalue = i.datavalue.astype('float')
    i.geoname = i.geoname.str.replace(' ','')
    i.geoname = i.geoname.str.lower()
    i.geoname = i.geoname.str.replace('-','')
    i[['name','state_abbrv']] = i['geoname'].str.split(',',1,expand=True)
    i['comp_us_%'] = ((i['datavalue'] - i.datavalue[0])/i.datavalue[0])*100
    i['comp_avg_%'] = ((i['datavalue'] - i.datavalue.median())/i.datavalue.median())*100

,code,geofips,geoname,timeperiod,cl_unit,unit_mult,datavalue,noteref,name,state_abbrv,comp_us_%,comp_avg_%
0,CAINC30-190,998,unitedstates(metropolitanportion),2018,Thousands of dollars,3,8.157592e+09,None,unitedstates(metropolitanportion),None,0.000000,151958.809349
1,CAINC30-190,10180,"abilene,tx(metropolitanstatisticalarea)",2018,Thousands of dollars,3,3.399340e+06,None,abilene,tx(metropolitanstatisticalarea),-99.958329,-36.635761
2,CAINC30-190,10420,"akron,oh(metropolitanstatisticalarea)",2018,Thousands of dollars,3,1.734578e+07,None,akron,oh(metropolitanstatisticalarea),-99.787366,223.328085
3,CAINC30-190,10500,"albany,ga(metropolitanstatisticalarea)",2018,Thousands of dollars,3,2.727856e+06,None,albany,ga(metropolitanstatisticalarea),-99.966561,-49.152329
4,CAINC30-190,10540,"albanylebanon,or(metropolitanstatisticalarea)",2018,Thousands of dollars,3,2.220171e+06,None,albanylebanon,or(metropolitanstatisticalarea),-99.972784,-58.615659
...,...,...,...,...,...,...,...,...,...,...,...,...
380,CAINC30-190,49420,"yakima,wa(metropolitanstatisticalarea)",2018,Thousands of dollars,3,4.654079e+06,None,yakima,wa(metropolitanstatisticalarea),-99.942948,-13.247226
381,CAINC30-190,49620,"yorkhanover,pa(metropolitanstatisticalarea)",2018,Thousands of dollars,3,9.404978e+06,None,yorkhanover,pa(metropolitanstatisticalarea),-99.884709,75.310289
382,CAINC30-190,49660,"youngstownwarrenboardman,ohpa(metropolitanstat...",2018,Thousands of dollars,3,9.373598e+06,None,youngstownwarrenboardman,ohpa(metropolitanstatisticalarea),-99.885094,74.725361
383,CAINC30-190,49700,"yubacity,ca(metropolitanstatisticalarea)",2018,Thousands of dollars,3,2.660595e+06,None,yubacity,ca(metropolitanstatisticalarea),-99.967385,-50.406085


In [402]:
#Join all the df from the API together.
mdf = df.join(df1.set_index('geoname'),on='geoname',lsuffix='_df',rsuffix='_df1')
mdf = mdf.join(df2.set_index('geoname'),on='geoname',rsuffix='_df2')
mdf = mdf.join(df3.set_index('geoname'),on='geoname',rsuffix='_df3')
mdf.columns
mdf['gdp_per_c'] = mdf['datavalue_df'] / mdf['datavalue_df1'] #Now I have gdp per capita for each MSA.

mdf['comp_us_%_gdp_per_c'] = ((mdf['gdp_per_c'] - mdf.gdp_per_c[0])/mdf.datavalue[0])*100
mdf['comp_avg_%_gdp_per_c'] = ((mdf['gdp_per_c'] - mdf.gdp_per_c.median())/mdf.datavalue.median())*100
mmdf = mdf.join(dfeduc.set_index('name'),on='name',rsuffix='_educ')
mmdf

,code_df,geofips_df,geoname,timeperiod_df,cl_unit_df,unit_mult_df,datavalue_df,noteref_df,name_df,state_abbrv_df,comp_us_%_df,comp_avg_%_df,code_df1,geofips_df1,timeperiod_df1,cl_unit_df1,unit_mult_df1,datavalue_df1,noteref_df1,name_df1,state_abbrv_df1,comp_avg_%_df1,comp_us_%_df1,code,geofips,timeperiod,cl_unit,unit_mult,datavalue,noteref,name,state_abbrv,comp_us_%,comp_avg_%,code_df3,geofips_df3,timeperiod_df3,cl_unit_df3,unit_mult_df3,datavalue_df3,noteref_df3,name_df3,state_abbrv_df3,comp_us_%_df3,comp_avg_%_df3,gdp_per_c,comp_us_%_gdp_per_c,comp_avg_%_gdp_per_c,ba+,other,population_age_25+_with_at_least_a_bachelor's_degree_(2017),share_of_population_age_25+_with_at_least_a_bachelor's_degree,share_(rank_out_of_388),percentage_point_increase:_share_of_population_age_25+_with_at_least_a_bachelor's_degree_(2007-2017),change_(rank_out_of_388),2007_ba+,2012_ba+,2017_ba+,state_abbrv_educ
0,CAGDP1-3,998,unitedstates(metropolitanportion),2018,Thousands of dollars,3,1.841535e+10,None,unitedstates(metropolitanportion),None,0.000000,146634.564988,CAINC30-100,998.0,2018.0,Number of persons,0.0,281421381.0,None,unitedstates(metropolitanportion),None,112247.004108,0.000000,CAINC30-190,998.0,2018.0,Thousands of dollars,3.0,8.157592e+09,None,unitedstates(metropolitanportion),None,0.000000,151958.809349,CAINC30-300,998.0,2018.0,Dollars,0.0,59609.0,None,unitedstates(metropolitanportion),None,0.000000,26.531522,65.436939,0.000000e+00,0.000282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CAGDP1-3,10180,"abilene,tx(metropolitanstatisticalarea)",2018,Thousands of dollars,3,7.276717e+06,None,abilene,tx(metropolitanstatisticalarea),-99.960486,-42.018713,CAINC30-100,10180.0,2018.0,Number of persons,0.0,171138.0,None,abilene,tx(metropolitanstatisticalarea),-31.679528,-99.939188,CAINC30-190,10180.0,2018.0,Thousands of dollars,3.0,3.399340e+06,None,abilene,tx(metropolitanstatisticalarea),-99.958329,-36.635761,CAINC30-300,10180.0,2018.0,Dollars,0.0,44010.0,None,abilene,tx(metropolitanstatisticalarea),-26.168867,-6.580344,42.519587,-2.809328e-07,-0.000145,0.215,0.785,22874.495,21.5,290.0,1.4,345.0,20.1,21.6,21.5,TX
2,CAGDP1-3,10420,"akron,oh(metropolitanstatisticalarea)",2018,Thousands of dollars,3,3.707062e+07,None,akron,oh(metropolitanstatisticalarea),-99.798697,195.380767,CAINC30-100,10420.0,2018.0,Number of persons,0.0,703818.0,None,akron,oh(metropolitanstatisticalarea),180.973121,-99.749906,CAINC30-190,10420.0,2018.0,Thousands of dollars,3.0,1.734578e+07,None,akron,oh(metropolitanstatisticalarea),-99.787366,223.328085,CAINC30-300,10420.0,2018.0,Dollars,0.0,50009.0,None,akron,oh(metropolitanstatisticalarea),-16.104951,6.153683,52.670747,-1.564946e-07,0.000044,0.308,0.692,148040.200,30.8,126.0,3.0,221.0,27.8,28.3,30.8,OH
3,CAGDP1-3,10500,"albany,ga(metropolitanstatisticalarea)",2018,Thousands of dollars,3,5.738808e+06,None,albany,ga(metropolitanstatisticalarea),-99.968837,-54.272858,CAINC30-100,10500.0,2018.0,Number of persons,0.0,147919.0,None,albany,ga(metropolitanstatisticalarea),-40.948849,-99.947439,CAINC30-190,10500.0,2018.0,Thousands of dollars,3.0,2.727856e+06,None,albany,ga(metropolitanstatisticalarea),-99.966561,-49.152329,CAINC30-300,10500.0,2018.0,Dollars,0.0,42194.0,None,albany,ga(metropolitanstatisticalarea),-29.215387,-10.435152,38.796963,-3.265667e-07,-0.000214,0.195,0.805,19459.830,19.5,331.0,1.6,330.0,17.9,16.1,19.5,GA
3,CAGDP1-3,10500,"albany,ga(metropolitanstatisticalarea)",2018,Thousands of dollars,3,5.738808e+06,None,albany,ga(metropolitanstatisticalarea),-99.968837,-54.272858,CAINC30-100,10500.0,2018.0,Number of persons,0.0,147919.0,None,albany,ga(metropolitanstatisticalarea),-40.948849,-99.947439,CAINC30-190,10500.0,2018.0,Thousands of dollars,3.0,2.727856e+06,None,albany,ga(metropolitanstatisticalarea),-99.966561,-49.152329,CAINC30-300,10500.0,2018.0,Dollars,0.0,42194.0,None,albany,ga(metropolitanstatisticalarea),-29.215387,-10.435152,38.796963,-3.265667e-07,-0.000214,0.186,0.814,15502.728,18.6,340.0,3.2,191.0,15.4,16.2,18.

In [437]:
#Multiple linear regression
mmdf.columns
data = mmdf
data1 = data[data['comp_us_%_gdp_per_c'].notnull() & data['ba+'].notnull() & data['comp_avg_%_df'].notnull() & data['comp_us_%_df3'].notnull()]
x = data1[['comp_us_%_gdp_per_c','ba+','comp_avg_%_df']]
y = data1[['comp_us_%_df3']]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, train_size = 0.70, test_size = 0.30, random_state = 100)

#mlr = LinearRegression()
#mlr.fit(X_train, y_train)

from sklearn.linear_model import LinearRegression 
regressor = LinearRegression() 
regressor.fit(X_train, y_train)

#Predicting the Test set result 
X_test['y_pred'] = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error 
print("The Mean Squared Error is- {}".format(mean_squared_error(y_test, X_test['y_pred']))) 
#sns.lineplot(x='X_test',y='y_pred')
print(X_train)

The Mean Squared Error is- 89.49723445226907
     comp_us_%_gdp_per_c    ba+  comp_avg_%_df
123        -3.592053e-07  0.211     -57.473172
351        -9.560479e-08  0.269     356.521507
24          9.726371e-08  0.388    1538.654943
31         -3.689447e-07  0.172     -66.962951
210        -2.391934e-07  0.160     -60.233386
..                   ...    ...            ...
336        -3.759973e-07  0.274     -62.688073
355         3.409486e-08  0.340     433.280425
327        -3.517370e-07  0.237     -62.940166
276        -5.643056e-08  0.388     159.921787
10         -1.633938e-07  0.234       9.627475

[269 rows x 3 columns]


<ipython-input-437-17c2cbb13977>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['y_pred'] = regressor.predict(X_test)
